In [645]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use("fivethirtyeight")

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Getting data

In [646]:
dataset_folder = 'datasets'

dataset_name_train = 'train.csv'
dataset_name_test = 'test.csv'
sample_submission = 'sample_submission.csv'

train = pd.read_csv(dataset_folder +'/'+ dataset_name_train)
test = pd.read_csv(dataset_folder +'/'+ dataset_name_test)
sample_submission = pd.read_csv(dataset_folder +'/'+ sample_submission)


X and y

In [647]:
X_train = train.drop(columns={'SalePrice', "Id"}, axis=1)
y_train = train['SalePrice']

X_test = test.drop(columns={"Id"}, axis=1)


Get num and cat features

In [648]:
num_features = X_train.select_dtypes(include=np.number).columns.tolist()
cat_features = X_train.select_dtypes(['object']).columns.tolist()

train[cat_features] = train[cat_features].astype('category')

In [649]:
print(cat_features)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


Delete duplicates

In [650]:
train.drop_duplicates(inplace=True)

In [651]:
cat_transformer_linear = OneHotEncoder(handle_unknown='ignore')
num_transformer_linear = Pipeline([ ('Imputer', SimpleImputer(strategy='median')), ('scaler', MinMaxScaler()) ])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer_linear, num_features),
        ("cat", cat_transformer_linear, cat_features)
    ]
)

In [652]:
# imputer = SimpleImputer(strategy='median')
# X_train[num_features] = imputer.fit_transform(X_train[num_features])

# #top models table
#     print('Top ' + name + ' models:')
#     results_df = pd.DataFrame(gs.cv_results_)
#     results_df = results_df.sort_values(by=["rank_test_score"])
#     display(results_df[['params', 'mean_test_score', 'mean_fit_time', 'rank_test_score']]
#             .head(5)
#             .rename(columns={'mean_test_score': 'cv_score'})
#             .rename(columns={'rank_test_score': 'rank_cv_score'}))

#     #best cv score
#     print('best cv score of ' + name + ' model:')
#     print(gs.best_score_)

#     #test score of best estimator + time prediction calculation
#     print('Test ' + name + ' score:')
#     start = time.time()
#     print(mean_squared_error(y_test, gs.best_estimator_.predict(X_test), squared=False))
#     pred_time = time.time() - start

#     #time prediction speed of best estimator
#     print('Prediction time best ' + name + ' model:')
#     print(pred_time)   
#     print()
#     print()     

# results = []
# for pipeline, param_grid, name in zip(pipelines, param_grids, names):   
#     results.append(gs_and_result(pipeline, param_grid, name))  


Result lists

In [653]:
pipelines = []
params = []
names = []

Pipelines

In [654]:
pipeline_tree = Pipeline([('preprocessor_linear', preprocessor), ('trun_svd', TruncatedSVD(n_components=20)),
                          ('tree', None)])

# pipeline_lgm = Pipeline([('preprocessor_linear', preprocessor), ('trun_svd', TruncatedSVD(n_components=10)),
#                           ('lgm', None)])    
   
pipeline_lgm = Pipeline([('lgm', None)])      

# pipeline_catboost = Pipeline([ ('cat', None)])                          

Parametres

In [655]:
param_tree = [
    {'tree': (RandomForestRegressor(random_state=1),),
     'tree__n_estimators': range(1, 20, 5),
     'tree__max_depth': range(1, 20, 5),
     }
]

param_tree_default = [
    {'tree': (RandomForestRegressor(random_state=1),),     
     }
]

param_lgm = [
    {'lgm': (LGBMRegressor(random_state=1, learning_rate=0.1, boosting_type='goss'),),
     'lgm__n_estimators': [50, 100, 200, 400, 800],
     'lgm__max_depth': [5, 10, 20, 40, 80],
     }
]

param_lgm_default = [
    {'lgm': (LGBMRegressor(random_state=1),),}
]

# param_catboost = [
#     {'cat': (CatBoostRegressor(random_state=1, cat_features=cat_features, nan_mode='Min'),),
#      }
# ]

In [656]:
names = ['TREE' , "TREE_DEFAULT", "LGM", "LGM_DEFAULT"]

In [657]:
pipelines.append(pipeline_tree)
pipelines.append(pipeline_tree)
pipelines.append(pipeline_lgm)
pipelines.append(pipeline_lgm)
# pipelines.append(pipeline_catboost)

params.append(param_tree)
params.append(param_tree_default)
params.append(param_lgm)
params.append(param_lgm_default)
# params.append(param_catboost)



In [658]:
results = []
for pipeline, param_grid, name in zip(pipelines, params, names):   
     gs = GridSearchCV(pipeline,
                  param_grid,
                  scoring='neg_root_mean_squared_error',
                  refit='neg_root_mean_squared_error',
                  n_jobs=10,
                  cv=5 )

     gs.fit(X_train, y_train)  
     
     #top models table
     print('Top ' + name + ' models:')
     results_df = pd.DataFrame(gs.cv_results_)
     results_df = results_df.sort_values(by=["rank_test_score"])
     display(results_df[['params', 'mean_test_score', 'mean_fit_time', 'rank_test_score']]
            .head(5)
            .rename(columns={'mean_test_score': 'cv_score'})
            .rename(columns={'rank_test_score': 'rank_cv_score'}))


Top TREE models:


,params,cv_score,mean_fit_time,rank_cv_score
11,"{'tree': RandomForestRegressor(max_depth=11, n...",-41192.953387,0.528223,1
10,"{'tree': RandomForestRegressor(max_depth=11, n...",-41725.778234,0.477052,2
14,"{'tree': RandomForestRegressor(max_depth=11, n...",-42071.345960,0.498200,3
15,"{'tree': RandomForestRegressor(max_depth=11, n...",-42389.013177,0.481799,4
7,"{'tree': RandomForestRegressor(max_depth=11, n...",-42790.655162,0.469201,5


Top TREE_DEFAULT models:


,params,cv_score,mean_fit_time,rank_cv_score
0,{'tree': RandomForestRegressor(random_state=1)},-40442.194344,1.197199,1


Top LGM models:


,params,cv_score,mean_fit_time,rank_cv_score
1,"{'lgm': LGBMRegressor(boosting_type='goss', ma...",-40513.562330,0.410199,1
16,"{'lgm': LGBMRegressor(boosting_type='goss', ma...",-40746.340000,0.436199,2
21,"{'lgm': LGBMRegressor(boosting_type='goss', ma...",-40784.213218,0.419200,3
15,"{'lgm': LGBMRegressor(boosting_type='goss', ma...",-40823.794002,0.404800,4
20,"{'lgm': LGBMRegressor(boosting_type='goss', ma...",-40837.208194,0.387799,5


Top LGM_DEFAULT models:


,params,cv_score,mean_fit_time,rank_cv_score
0,{'lgm': LGBMRegressor(random_state=1)},-41673.792422,0.380798,1
